In [1]:
"""
Testing models and deciding on a final model
"""

# TODO: Test more models - RandomForestRegressor, XGBRegressor, CatBoostRegressor, LGBMRegressor
# TODO: Stacking of some kind (combine all - many examples in github).

''

In [1]:
import pandas as pd
import numpy as np 
import json
import ast
from collections import Counter
import xgboost as xgb
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
# pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
DATA_FOLDER = "/StudentData/hw1/"

In [3]:
X_train = pd.read_csv(DATA_FOLDER + "X_train.csv")
y_train = pd.read_csv(DATA_FOLDER + "y_train.csv")
X_train

,budget,popularity,vote_average,vote_count,belongs_to_collection_A Nightmare on Elm Street Collection,belongs_to_collection_Friday the 13th Collection,belongs_to_collection_Halloween Collection,belongs_to_collection_James Bond Collection,belongs_to_collection_One Piece Collection,belongs_to_collection_Police Academy Collection,belongs_to_collection_Saw Collection,belongs_to_collection_The Pink Panther (Original) Collection,belongs_to_collection_The Terminator Collection,genres_Action,genres_Adventure,genres_Animation,genres_Comedy,genres_Crime,genres_Documentary,genres_Drama,genres_Family,genres_Fantasy,genres_History,genres_Horror,genres_Music,genres_Mystery,genres_Romance,genres_Science Fiction,genres_TV Movie,genres_Thriller,genres_War,genres_Western,production_companies_2,production_companies_4,production_companies_5,production_companies_7,production_companies_12,production_companies_14,production_companies_21,production_companies_25,production_companies_33,production_companies_34,production_companies_60,production_companies_79,production_companies_104,production_companies_174,production_companies_559,production_companies_694,production_companies_1632,production_companies_7295,production_companies_9195,production_companies_10163,production_countries_Australia,production_countries_Belgium,production_countries_Canada,production_countries_China,production_countries_Denmark,production_countries_France,production_countries_Germany,production_countries_Hong Kong,production_countries_India,production_countries_Ireland,production_countries_Italy,production_countries_Japan,production_countries_Mexico,production_countries_Netherlands,production_countries_Russia,production_countries_South Korea,production_countries_Spain,production_countries_Sweden,production_countries_United Kingdom,production_countries_United States of America,spoken_languages_ar,spoken_languages_cn,spoken_languages_cs,spoken_languages_de,spoken_languages_en,spoken_languages_es,spoken_languages_fr,spoken_languages_he,spoken_languages_hi,spoken_languages_hu,spoken_languages_it,spoken_languages_ja,spoken_languages_ko,spoken_languages_la,spoken_languages_pl,spoken_languages_pt,spoken_languages_ru,spoken_languages_sv,spoken_languages_tr,spoken_languages_zh,cast_Alec Baldwin,cast_Brad Pitt,cast_Bruce Willis,cast_Dennis Quaid,cast_Frank Welker,cast_J.K. Simmons,cast_John Goodman,cast_John Leguizamo,cast_Keith David,cast_Liam Neeson,cast_Matt Damon,cast_Morgan Freeman,cast_Nicolas Cage,cast_Richard Jenkins,cast_Robert De Niro,cast_Robin Williams,cast_Samuel L. Jackson,cast_Steve Buscemi,cast_Tommy Lee Jones,cast_Willem Dafoe,crew_Avy Kaufman,crew_Barbara Harris,crew_Bob Weinstein,crew_Clint Eastwood,crew_Dan O'Connell,crew_Dan Perri,crew_Deborah Aquila,crew_Gary Burritt,crew_Hans Zimmer,crew_Harvey Weinstein,crew_James Horner,crew_James Newton Howard,crew_John T. Cucci,crew_Kerry Barden,crew_Luc Besson,crew_Mary Vernieu,crew_Mo Henry,crew_Nerses Gezalyan,crew_Robert Rodriguez,crew_Steven Spielberg,original_language_ab,original_language_af,original_language_ar,original_language_bn,original_language_cn,original_language_cs,original_language_da,original_language_de,original_language_el,original_language_en,original_language_es,original_language_fa,original_language_fi,original_language_fr,original_language_he,original_language_hi,original_language_hu,original_language_id,original_language_it,original_language_ja,original_language_ka,original_language_ko,original_language_mi,original_language_ml,original_language_nb,original_language_nl,original_language_no,original_language_pl,original_language_pt,original_language_ro,original_language_ru,original_language_sk,original_language_sl,original_language_sv,original_language_sw,original_language_ta,original_language_te,original_language_th,original_language_tl,original_language_tr,original_language_vi,original_language_xx,original_language_zh,has_homepage,has_video,title_length,overview_length,tagline_length,num_belongs_to_collecti

In [7]:
def plot_prediction_stats(X_valid,y_valid,y_pred):
    # see where the model is failing so we can think of ways to improve it.
    pass

def adjust_prediction(pred):
    pred = np.e ** pred - 1
    pred[pred<0] = 0
    return pred


def cross_validation(model,X,y,num_folds,scaler):
    folds = KFold(n_splits=num_folds, shuffle=True)
    for k, (train_indexes, valid_indexes) in enumerate(folds.split(X,y)):
        X_train, y_train = X.iloc[train_indexes], y.iloc[train_indexes]
        X_valid, y_valid = X.iloc[valid_indexes], y.iloc[valid_indexes]
#         X_train = scaler.transform(X_train)
#         X_valid = scaler.transform(X_valid)
        y_train = np.log(y_train + 1)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_valid)
        y_pred = adjust_prediction(y_pred)
        print("RMLSE on fold {}: {}".format(k+1,np.sqrt(mean_squared_log_error(y_valid, y_pred))))
        plot_prediction_stats(X_valid,y_valid,y_pred)

In [8]:
# model_xgb = xgb.XGBRegressor(max_depth=5, 
#                             learning_rate=0.01, 
#                             n_estimators=10000, 
#                             objective='reg:linear', 
#                             gamma=1.45, 
#                             silent=True,
#                             subsample=0.8, 
#                             colsample_bytree=0.7, 
#                             colsample_bylevel=0.5)
scaler = MinMaxScaler().fit(X_train)
model_xgb = xgb.XGBRegressor()
cross_validation(model_xgb,X_train,y_train,5,scaler)

RMLSE on fold 1: 2.243493376701017
RMLSE on fold 2: 2.1665936621008015
RMLSE on fold 3: 2.353629823639398
RMLSE on fold 4: 2.482472336462375
RMLSE on fold 5: 2.2613673013492757


array([[0.97623002, 0.13695748, 0.62      , ..., 0.06      , 0.1875    ,
        0.135     ],
       [0.55024641, 0.02162065, 0.43      , ..., 0.01578947, 0.15833333,
        0.07      ],
       [0.        , 0.02155385, 0.65      , ..., 0.06      , 0.24444444,
        0.12888889],
       ...,
       [0.8441386 , 0.04292737, 0.56      , ..., 0.01684211, 0.06666667,
        0.2       ],
       [0.81924808, 0.12138821, 0.81      , ..., 0.02315789, 0.14166667,
        0.0725    ],
       [0.75025277, 0.06274463, 0.57      , ..., 0.00315789, 0.19761905,
        0.02857143]])